In [20]:
import pandas as pd
import json

In [ ]:
df = pd.read_json('resources/result.json')

In [ ]:
# Global parameters
TARGET_CHAT_NAME = ''

In [ ]:
# Get specific telegram chat data
chats = df['chats']['list']
chat_1 = next((chat for chat in chats if chat.get('name') == TARGET_CHAT_NAME), None)

if chat_1:
    chat_display = {k: v for k, v in chat_1.items() if k != 'messages'}
    chat_display["message_count"] = len(chat_1.get("messages", []))
    print(f"Chat <{TARGET_CHAT_NAME}>: {json.dumps(chat_display, ensure_ascii=False, indent=2)}")
else:
    print(f"Chat <{TARGET_CHAT_NAME}> not found.")

Chat <Айша>: {
  "name": "Айша",
  "type": "personal_chat",
  "id": 857009286,
  "message_count": 4814
}
